In [2]:
# Import relevant packages

# Import os to utilize the built in functionality like the current working directory
import os
# For calculations
import numpy as np
# Import pandas to utilize dataframes and to read the xlsx files
import pandas as pd
# For pathnames
import glob
#For unlisting
import itertools
#For using math function like sqrt
import math 
# Find out the current working directory
#print(os.getcwd())
#os.chdir('Your file path') #use to set the path to working directory


In [2]:
class Preprocess():
    def __init__(self):
        unique_processed_links= [] 
        
    def loadfile(self, file_path):
        """Read the file and load the data

        Parameters
        ----------
         file_path: str
            A valid file path and file name contianing the data.
            Returns
        -------
        Panda Series
            It returns Link column of the file

        """
   
        df = pd.read_csv(file_path,  error_bad_lines=False)
        return(df['Link'])

    
    def removeselflinks(self, file_path):
        """Remove the self links and extracts only the outlinks.
           The links are preprocessed to short name eg. https://www.uu.se/contact 
           will be converted to uu.se. 
           A web page can outlink to another page more than once, so duplicates will
           be removed.

        Parameters
        ----------
         file_path: path and name of file to be read
            
       
        """
        raw_data = self.loadfile(file_path)
        #df= pd.read_csv(file_path,  error_bad_lines=False)
        #raw_data= df['Link']
        
        unique_raw_data = []  
        for i in raw_data:
            if i.find('wikipedia.org')== -1:  #Check if a link is selflink: Files were generated from
                                          #wikipedia, therefore a link contianing 'wikipedia.org'
                                          #represents the inlink and is removed.
                if i.find('/', 8)!=1:  # Check if outlink has long (e-g:https://www.uu.se/contact ) 
                                   # or short (https://www.uu.se) format
                    intermediate_name=i[0: i.find('/', 8)]
                else:
                    intermediate_name=i
             
                #Remove http:// or https:// etc.
                intermediate_name=intermediate_name[ (intermediate_name.find('//' )+2):len(intermediate_name) ]
            
                #Some addresses are without www. To, keep the same format, www is removed
                if intermediate_name.find('www')!=-1: 
                    intermediate_name= intermediate_name[ (intermediate_name.find('www')+4):len(intermediate_name) ]  
                #remove empty links
                if len(intermediate_name)>0: 
                    unique_raw_data.append(intermediate_name)  
                #Remove duplicates
                self.unique_processed_links=pd.unique(unique_raw_data)
               
  


In [3]:
class PopulateDictionaries(Preprocess):
        def __init__(self):
            self.pages={} # Create a dictionary of pages
            self.pageindex=0 #To keep track of index of the page
            self.inlink_dict={} #a dictionary that record in_links with pagename, pageindex, mainpageindex (page where link originated), and update mainpage indeces, if it was inlinked from more than one pages 
            self.outlink_dict={} #a dictionary that record out_links with pagename, pageindex, outlinkindex (page where link is directed to), and update mainpage indeces, if it was out bound from more than one pages 
            Preprocess.__init__(self)
        
        def addpages(self, list_pages):
            
            """Add pages to a global dictionary of pages and index them 
       

            Parameters
            ----------
             list_pages: list
                A processed list of all pages in a data file
        
            """

            #global pages
            #global pageindex
            for ind, page_name in enumerate(list_pages):
                if self.pages.get(page_name)==None: # Check for duplicates
                    self.pageindex=self.pageindex+1
                    self.pages[page_name]=[self.pageindex]

        def inlinkgraph(self, out_links):
    
            """Creates dictionary of inlink graph that records in_links with pagename, pageinde and mainpageindex,
                If a webpage is inlinked from more than one main pages then indeces are updated.
                For example: Consider a entery in link_dict is  'usnews.com': [[10], [1], [44]], 
                Then, webpage usnews.com has index 10 and inlinked by main pages 1 and 44.

            Parameters
            ----------
             out_links: list
                A processed list of all pages in a data file
        
          
            """
            for ind, pname in enumerate(out_links):
                #Check if a page already exists in link_dict
                if self.inlink_dict.get(pname)==None: # If a page is not present is dict
                    self.inlink_dict[pname]= [ self.pages[pname], self.pages[out_links[0]] ] # Add the page and indeces
                else: #A page already exists in the inlink_dict, update the main page indeces
                    self.inlink_dict[pname]= [  list(itertools.chain(*self.inlink_dict[pname])), self.pages[out_links[0]]]


        def outlinkgraph(self, out_links):
            """Creates dictionary of link graph that records out_links with pagename, pageinde and mainpageindex,
                If a webpage has many out linkes then indeces are updated.
                For example: Consider a entery in link_dict is  'usnews.com': [[10], [1], [44]], 
                Then, webpage usnews.com has index 10 and inlinked by main pages 1 and 44.

                Parameters
                ----------
                 out_links: list
                    A processed list of all pages in a data file
                
            """
            for ind, pname in enumerate(out_links):
            #global pages
            #global outlink_dict
            #Check if a page already exists in link_dict
                if self.outlink_dict.get(out_links[0])==None: # If a page is already present in the dict
                    self.outlink_dict[pname]= [ self.pages[pname] ]
                    # Add the first page and its index
             
                else:#A page already exists in the outlink_dict, update the inlink page indeces
                    self.outlink_dict[out_links[0]]= [  list(itertools.chain(*self.outlink_dict[out_links[0]])), self.pages[pname]]



In [4]:
class AdjacencyMatrices(PopulateDictionaries):
    def __init__(self):
        self.adj_m_pagerank=None #Initialise the adjacancey matrix for pagerank algo.
        self.adj_m_HITS=None    #Initialise the adjacancey matrix for HITS algo.
        PopulateDictionaries.__init__(self) 
    
###############Create Adjacency matrix Page rank
    def adjpagerank(self, dict_inlinks):
        """Adjacacy matrix for page rank algo:  
                        #An Adjacency matrix of in links of web pages divided by total number of out links of a page.
                        #Each element of A that is A_i,j  represents the out link from web page 'i' (row) to web page 'j' (column).
                        #Alternatively,  We can also say that in link from web page 'j' (column) to web page 'i' (row).
                        #Note that  for all 'i' sum(i, M_i,j) = 1 and A must be a square matrix.
                        
        Parameters
        ----------
       
        dict_inlinks : dictionary
               A dictionary of in links
    
    
        
        """
        zero_data = np.zeros(shape=(len(dict_inlinks),len(dict_inlinks)))
        self.adj_m_pagerank = pd.DataFrame(zero_data)

        for i in dict_inlinks:
            
            link_map=(list(itertools.chain(*dict_inlinks[i])))
            for ind, j in enumerate(link_map):
                if ind!=0: # It is the page index, so need both the page index and main page index.
            
                    #If a page index and main page index is similar then it is self link and is removed.
                    if link_map[ind]!=link_map[0]:
                        self.adj_m_pagerank.iat[ link_map[ind]-1, (link_map[0]-1)  ]='1'
                        
        ###########divide the 1 by the total out links (# Only divide if row sum is not 0)
        self.adj_m_pagerank=self.adj_m_pagerank.apply(lambda x : x.div(x.sum()) if (x.sum()!=0) else 0 , axis=1) 


###############Create Adjacency matrix FOR HITS
    def adjHITS(self, dict_outlinks):
        """Adjacacy matrix for HITS algo:  
                        #An Adjacency matrix of out links of web pages.
                        #Each element of L that is L_i,j  represents the out link from web page 'i' (row) to web page 'j' (column).
                        #Note L must be a square matrix.
                        
        Parameters
        ----------
       
        dict_inlinks : dictionary
               A dictionary of out links
    
        """


        zero_data = np.zeros(shape=(len(dict_outlinks),len(dict_outlinks)))
        self.adj_m_HITS = pd.DataFrame(zero_data)
        for i in dict_outlinks:
            link_map=(list(itertools.chain(*dict_outlinks[i])))
            for ind, j in enumerate(link_map):
                if ind!=0: # It is the page index, so need both the page index and main page index.
                #If a page index and main page index is similar then it is self link and is removed.
                    if link_map[ind]!=link_map[0]:
                        self.adj_m_HITS.iat[ (link_map[0]-1) , link_map[ind]-1  ]='1'


In [116]:
class PagerankAlgo():
    def __init__(self, A, d):
        self.d= d       #Teleporting parameter
        
        self.A= A       #An Adjacency matrix of in links of web pages divided by total number of out links of a page.
                        #Each element of A that is A_i,j  represents the out link from web page 'i' (row) to web page 'j' (column).
                        #Note that  for all 'i' sum(i, M_i,j) = 1 and A must be a square matrix.
        
        self.P= np.ones(len(self.A)) #Intial page rank =1


    def calc_pagerank(self, max_itrs):
        
        """PageRank Algorithm:  This algorithm was propsed by the Larry Page and Sergey Brin at Stanford University 
                            and it ranks the web pages by measuring their importance.
                            It is used by the search engine Google.
        Parameters
        ----------
       
        max_itrs : int
               Max number of iterations
    
    
        Returns
        -------
        numpy array
            A vector of ranks such that p_i is the i-th rank in the range of [0, 1].
    
        Note
        -----
            1) Don't forget to normalize the page rank values in each iteration by max of page rank value.
               This is done to restrict the page rank values in the range of 1-0.
            2) Finally, normalize the page ranks by the sum of values of page ranks. This is only done at the 
               final calculation.
               This is done to so that sum of final page ranks =1.
        """
        #Check if A is square matrix
        assert(self.A.shape[0]==self.A.shape[1])
        n = self.A.shape[0]
    
        P_r = ((1-self.d)/n)*np.ones((n,n)) + (self.d)*self.A   #Apply page rank equation
        Pr_T = P_r.T
        P_t = self.P.copy()
        for i in range(max_itrs):
            y = Pr_T@P_t
            P_t = y/np.max(y)
            #print('Results of Iteration {}\n'.format(i),(P_t/np.sum(P_t) )[0:25])
            #print('Max value of iteration {} is {}\n'.format(i,max(P_t/np.sum(P_t) )))
        page_ranks= P_t/np.sum(P_t)    
        return(page_ranks)



In [138]:
class HITSalgo():
    def __init__(self, L):
        self.L= L   #An Adjacency matrix of out links of web pages 
                    #Each element of L that is L_i,j  represents the out link from web page 'i' to web page 'j'.
                    #Note that L must be a square matrix.
                    
        self.a= np.ones(len(L)) #Initial authority values =1
        self.h= np.ones(len(L)) #Initial hub values =1



    def calc_HITS(self, max_itrs):
        """HITS Algorithm:  HITS algorithm was propsed by Jon M. Lleinberg  at Cornell University 
                            and it ranks the web pages by measuring their authoraty and hubs.
                            It is used by the search engine Ask.
        Parameters
        ----------
        max_itrs : int
               Max number of iterations
    
   
        Returns
        -------
        Panda series
            A series conisting of normalized authority and  hub scores.
    
        Note
        -----
        1) Don't forget to normalize the authority score by the sum of sequare values of all authority score. 
        2) Don't forget to normalize the hub score by the sum of sequare values of all hub score.
        """
       
        #Check if adjacency matrix is a square matrix
        assert(self.L.shape[0]==self.L.shape[1])
        n = self.L.shape[0]
        L_T= self.L.T   
        pr_h= self.L@L_T 
       
        pr_a=L_T@self.L
   
    
        a_cal=self.a
        h_cal=self.h
        for i in range(max_itrs):
            norm=0
            #print ('i', i)
            a_cal=pr_a@a_cal
            #print('non-normalized a_cal', a_cal)
            norm= sum([x ** 2 for x in a_cal])
            a_cal=a_cal/ math.sqrt(norm)
            #print("Normalized a_cal:", sorted( a_cal[0:17], reverse=True) )
        
        
            norm=0
            h_cal=pr_h@h_cal
            norm= sum([x ** 2 for x in h_cal])
            h_cal=h_cal/ math.sqrt(norm)
            #print('Normalized h_cal', sorted( h_cal[0:17], reverse=True) )
        return(a_cal, h_cal)


In [5]:
#MAIN of the code, #if __name__ == '__main__'
pp_data = AdjacencyMatrices()

file_list =  glob.glob('DataMining2022/Pagerank_assignment/Data_files' + "/*.csv")

for fl in file_list:
   
    pp_data.removeselflinks(fl)
    list_out_links= pp_data.unique_processed_links.tolist()
    pp_data.addpages(list_out_links)

print(pp_data.pages)

{'wolffund.org.il': [1], 'leidenranking.com': [2], 'circ.ahajournals.org': [3], 'upsalafaktning.se': [4], 'britannica.com': [5], 'bbc.co.uk': [6], 'retractionwatch.com': [7], 'ec.europa.eu': [8], 'uu.se': [9], 'tsl.uu.se': [10], 'thelocal.se': [11], 'idref.fr': [12], 'creativecommons.org': [13], 'authority.bibsys.no': [14], 'catalogue.bnf.fr': [15], 'isa.org': [16], 'getty.edu': [17], 'lundagard.se': [18], 'aleph.nkp.cz': [19], 'ki.se': [20], 'jstor.org': [21], 'data.nlg.gr': [22], 'usnews.com': [23], 'cantic.bnc.cat': [24], 'd-nb.info': [25], 'geo.uu.se': [26], 'ci.nii.ac.jp': [27], 'cwur.org': [28], 'euroscholars.eu': [29], 'lub.lu.se': [30], 'opac.vatlib.it': [31], 'ep.liu.se': [32], 'en.wiktionary.org': [33], 'sfv.se': [34], 'campusgotland.uu.se': [35], 'nytimes.com': [36], 'dagensmedicin.se': [37], 'aop.se': [38], 'matarikinetwork.org': [39], 'developer.wikimedia.org': [40], 'ergo.nu': [41], 'mbl.is': [42], 'chiropractorswarwick.co.uk': [43], 'worldcat.org': [44], 'nla.gov.au': [4

In [36]:
#MAIN of the code, #if __name__ == '__main__'
pp_data = AdjacencyMatrices()

file_list =  glob.glob('DataMining2022/Pagerank_assignment/Data_files' + "/*.csv")

for fl in file_list:
   
    pp_data.removeselflinks(fl)
    list_out_links= pp_data.unique_processed_links.tolist()
    pp_data.addpages(list_out_links)
    pp_data.inlinkgraph(list_out_links)
    pp_data.outlinkgraph(list_out_links)
    
print(pp_data.pages)
print('In Links: \n', pp_data.inlink_dict)
print('Out Links:\n', pp_data.outlink_dict ) 
print(' \n')

#AdjacencyMatrices
##.  PageRank
pp_data.adjpagerank(pp_data.inlink_dict)
print("PageRankAdjMatr \n", pp_data.adj_m_pagerank )
print( 'If the rows are summing up to one in  adj_m_pagerank:\n',pp_data.adj_m_pagerank.sum(axis=1)[0:25])
print( 'Number of in-links in adj_m_pagerank: \n',np.count_nonzero(pp_data.adj_m_pagerank, axis=0)[0:25])
print( 'Number of out-links in adj_m_pagerank: \n',np.count_nonzero(pp_data.adj_m_pagerank, axis=1)[0:25])

##. HITS
pp_data.adjHITS(pp_data.outlink_dict)
print("HITSAdjMatr \n", pp_data.adj_m_HITS )
# Number of outlinks in adj_m_HITS
print( 'Number of outlinks in adj_m_HITS:\n',pp_data.adj_m_HITS.sum(axis=1)[0:25])


{'wolffund.org.il': [1], 'leidenranking.com': [2], 'circ.ahajournals.org': [3], 'upsalafaktning.se': [4], 'britannica.com': [5], 'bbc.co.uk': [6], 'retractionwatch.com': [7], 'ec.europa.eu': [8], 'uu.se': [9], 'tsl.uu.se': [10], 'thelocal.se': [11], 'idref.fr': [12], 'creativecommons.org': [13], 'authority.bibsys.no': [14], 'catalogue.bnf.fr': [15], 'isa.org': [16], 'getty.edu': [17], 'lundagard.se': [18], 'aleph.nkp.cz': [19], 'ki.se': [20], 'jstor.org': [21], 'data.nlg.gr': [22], 'usnews.com': [23], 'cantic.bnc.cat': [24], 'd-nb.info': [25], 'geo.uu.se': [26], 'ci.nii.ac.jp': [27], 'cwur.org': [28], 'euroscholars.eu': [29], 'lub.lu.se': [30], 'opac.vatlib.it': [31], 'ep.liu.se': [32], 'en.wiktionary.org': [33], 'sfv.se': [34], 'campusgotland.uu.se': [35], 'nytimes.com': [36], 'dagensmedicin.se': [37], 'aop.se': [38], 'matarikinetwork.org': [39], 'developer.wikimedia.org': [40], 'ergo.nu': [41], 'mbl.is': [42], 'chiropractorswarwick.co.uk': [43], 'worldcat.org': [44], 'nla.gov.au': [4

HITSAdjMatr 
      0    1    2    3    4    5    6    7    8    9    ...  124  125  126  \
0    0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  ...  1.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  0.0   
3    1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  ...  1.0  1.0  0.0   
4    0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  ...  1.0  0.0  1.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
129  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
130  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
131  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
132  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  0.0   
133  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0  ...  0.0  1.0  1.0   

     127  128  129  130  131  132  133  
0    0.0  0.0  0.0  

In [147]:
pr= PagerankAlgo(pp_data.adj_m_pagerank , 0.85)
page_rank_score=pr.calc_pagerank(50)
print('Highest page rank score is:', max(page_rank_score))
print('Page rank scores:\n', page_rank_score)

type(page_rank_score)

Highest page rank score is: 0.028857517404183585
Page rank scores:
 0      0.005859
1      0.009234
2      0.005798
3      0.010142
4      0.005699
         ...   
129    0.004581
130    0.005578
131    0.004837
132    0.004818
133    0.003552
Length: 134, dtype: float64


pandas.core.series.Series

In [146]:
hr= HITSalgo(pp_data.adj_m_HITS )
HITS_scores=hr.calc_HITS(5)
print('Highest authority score:', max( HITS_scores[0] ))
print('Authority scores:\n',  HITS_scores[0] )

print('Highest hub:', max( HITS_scores[1]))
print('Hub scores:\n',  HITS_scores[1] )

Highest authority score: 0.23539820221537977
Authority scores:
 0      0.075043
1      0.101172
2      0.061296
3      0.087153
4      0.055520
         ...   
129    0.050153
130    0.059948
131    0.060804
132    0.053635
133    0.030594
Length: 134, dtype: float64
Highest hub: 0.143257375709201
Hub scores:
 0      0.094271
1      0.108367
2      0.049826
3      0.118343
4      0.098623
         ...   
129    0.103270
130    0.057627
131    0.050051
132    0.073692
133    0.139772
Length: 134, dtype: float64
